# Initial cleaning of the *Lysekil* data

To summarize the notebook, the initial cleaning of the *Lysekil* data was performed. The notebook contains several cells that perform various tasks such as loading the data, basic preprocessing, removal of highly correlated data, renaming columns, and reordering the columns.

In [2]:
import sys
sys.path.append("../scripts")

import os
import pandas as pd
import load_data, preprocessing, visualize

# Set Pandas display options to show all columns
pd.set_option('display.max_columns', None)

## Loading data and basic cleaning

In [3]:
# Load data and use basic preprocessing
raw_df = load_data.lysekil()
df = preprocessing.initial_clean(raw_df)

# Print info and generate overview
visualize.print_df_info(df)
visualize.generate_overview(df)

Size of the DataFrame: 21894
Number of rows without NaN values: 21894
Number of rows without negative values: 21894


,Time,Downstream-Rich-FT1042,Downstream-Rich-AT1060,Downstream-Rich-TT1043,Upstream-Rich-TT1027,Upstream-Rich-PST1025,Desorber-Overhead-FT1067,Desorber-Overhead-PIC1032,Desorber-Overhead-TIC1062,Desorber-Overhead-Corrected,Desorber-Packing-PT1004,Desorber-Packing-TT1004,Desorber-Packing-PT1005,Desorber-Packing-TT1005,Desorber-Packing-TT1030B,Desorber-Packing-PT1035,Desorber-Sump-TT1029,Desorber-Sump-UX1029,Desorber-Sump-TIC1029,Desorber-Sump-Corrected,Upstream-Lean-TT1053,Downstream-Lean-FT1076,Downstream-Lean-AT1058,Downstream-Lean-TT1054
NaN count,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Valid count,NaN,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00
mean,NaN,1.83,1095.47,46.93,106.81,1.44,103.84,0.89,25.69,117.63,0.88,93.53,0.88,101.92,104.21,0.89,119.22,120.83,89.52,125.09,117.40,2.13,1027.80,71.37
std,NaN,0.32,6.17,3.15,1.96,0.10,9.11,0.08,6.30,9.79,0.08,3.71,0.08,1.95,3.55,0.08,1.76,10.63,6.90,9.52,1.59,0.27,7.39,5.36
min,NaN,0.07,1041.67,23.37,71.64,0.84,2.17,0.29,11.35,2.56,0.29,14.84,0.29,51.80,69.27,0.29,81.41,1.89,4.63,2.27,92.46,0.36,1009.22,30.04
25%,NaN,1.64,1090.60,44.94,105.99,1.43,100.71,0.90,18.89,114.12,0.90,92.29,0.90,101.34,102.67,0.90,118.96,115.44,85.99,120.24,117.28,1.98,1023.00,69.02
50%,NaN,1.81,1095.37,46.49,106.55,1.45,104.33,0.90,29.50,118.16,0.90,93.72,0.90,101.92,103.72,0.90,119.86,120.61,89.26,124.98,117.74,2.07,1027.15,72.46
75%,NaN,2.07,1099.68,48.35,108.12,1.49,108.29,0.90,30.17,123.31,0.90,95.11,0.90,102.79,105.82,0.90,120.01,128.12,93.94,131.77,118.06,2.31,1033.10,75.24
max,NaN,2.56,1114.45,61.03,112.46,2.24,134.04,1.50,46.28,146.72,1.50,113.58,1.50,116.80,118.53,1.50,127.56,136.50,105.00,140.55,125.38,2.93,1093.20,100.08


## Removal of highly correlated data

In [4]:
# Highly correlated columns to be removed
corr_cols = []

# Generate correlation matrices
pearson = df.iloc[:, 1:].corr(method="pearson")
spearman = df.iloc[:, 1:].corr(method="spearman")
kendall = df.iloc[:, 1:].corr(method="kendall")

# Generate heatmap
visualize.make_heatmap(pearson)

Looking at the heatmap above, we can determine some correlations:
* Desorber-Sump-UX1029 and Desorber-Sump-Corrected are highly correlated, we
  will therefore remove Desorber-Sump-UX1029 due to domain knowledge
* Desorber-Overhead-FT1067 and Desorber-Overhead-Corrected are highly 
  correlated, we will therefore remove Desorber-Overhead-FT1067 due to domain 
  knowledge
* Desorber-Sump-TIC1029 and Desorber-Sump-Corrected are highly correlated, we 
  will therefore remove Desorber-Sump-TIC1029 due to domain knowledge

In [5]:
# Print correlations
visualize.print_corr(df, "Desorber-Sump-UX1029", "Desorber-Sump-Corrected")
corr_cols.append("Desorber-Sump-UX1029")

visualize.print_corr(df, "Desorber-Overhead-FT1067", "Desorber-Overhead-Corrected")
corr_cols.append("Desorber-Overhead-FT1067")

visualize.print_corr(df, "Desorber-Sump-TIC1029", "Desorber-Sump-Corrected")
corr_cols.append("Desorber-Sump-TIC1029")


The correlation between the following columns
---------------------------------------------
Desorber-Sump-UX1029
Desorber-Sump-Corrected
ρ = 0.98
---------------------------------------------

The correlation between the following columns
---------------------------------------------
Desorber-Overhead-FT1067
Desorber-Overhead-Corrected
ρ = 0.93
---------------------------------------------

The correlation between the following columns
---------------------------------------------
Desorber-Sump-TIC1029
Desorber-Sump-Corrected
ρ = 0.99
---------------------------------------------


## Renaming columns and reordering
We will now reorder the columns so that they are sorted according to the PFD of the process.

In [6]:

# Remove highly correlated columns
drop_df = df.drop(corr_cols, axis=1)

# Rename columns
new_cols = ["F1", "D1", "T1",
            "T2", "P2",
            "P10", "T10", "F10",
            "P3", "T3",
            "P4", "T4",
            "T5", 
            "P6",
            "T7", "U7",
            "T8", "F8", "D8",
            "T9"]
old_cols = drop_df.columns.values[1:]

column_mapping = {old_col: new_col for old_col, new_col in zip(old_cols, new_cols)}
drop_df.rename(columns=column_mapping, inplace=True)

# Reordering of columns
order_cols = ["Time",
              "F1", "D1", "T1",
              "P2", "T2",
              "P3", "T3",
              "P4", "T4",
              "T5", 
              "P6",
              "T7", "U7",
              "F8", "D8", "T8", 
              "T9",
              "F10", "P10", "T10"]

new_df = drop_df[order_cols]

if not os.path.exists("../data/processed/renamed_Lysekil.pkl"):
    new_df.to_pickle("../data/processed/renamed_Lysekil.pkl")

visualize.generate_overview(new_df)

,Time,F1,D1,T1,P2,T2,P3,T3,P4,T4,T5,P6,T7,U7,F8,D8,T8,T9,F10,P10,T10
NaN count,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Valid count,NaN,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00,21894.00
mean,NaN,1.83,1095.47,46.93,1.44,106.81,0.88,93.53,0.88,101.92,104.21,0.89,119.22,125.09,2.13,1027.80,117.40,71.37,117.63,0.89,25.69
std,NaN,0.32,6.17,3.15,0.10,1.96,0.08,3.71,0.08,1.95,3.55,0.08,1.76,9.52,0.27,7.39,1.59,5.36,9.79,0.08,6.30
min,NaN,0.07,1041.67,23.37,0.84,71.64,0.29,14.84,0.29,51.80,69.27,0.29,81.41,2.27,0.36,1009.22,92.46,30.04,2.56,0.29,11.35
25%,NaN,1.64,1090.60,44.94,1.43,105.99,0.90,92.29,0.90,101.34,102.67,0.90,118.96,120.24,1.98,1023.00,117.28,69.02,114.12,0.90,18.89
50%,NaN,1.81,1095.37,46.49,1.45,106.55,0.90,93.72,0.90,101.92,103.72,0.90,119.86,124.98,2.07,1027.15,117.74,72.46,118.16,0.90,29.50
75%,NaN,2.07,1099.68,48.35,1.49,108.12,0.90,95.11,0.90,102.79,105.82,0.90,120.01,131.77,2.31,1033.10,118.06,75.24,123.31,0.90,30.17
max,NaN,2.56,1114.45,61.03,2.24,112.46,1.50,113.58,1.50,116.80,118.53,1.50,127.56,140.55,2.93,1093.20,125.38,100.08,146.72,1.50,46.28


# Further preprocessing and SRD

Making a new dataset that changes some key variables.

In [7]:
pearson = new_df[["P3", "P4", "P6", "P10"]].corr(method="pearson")
print(pearson)

           P3        P4        P6       P10
P3   1.000000  0.999790  0.999803  0.999814
P4   0.999790  1.000000  0.999817  0.999812
P6   0.999803  0.999817  1.000000  0.999889
P10  0.999814  0.999812  0.999889  1.000000


In [8]:
pearson = new_df[["T7", "T8"]].corr(method="pearson")
print(pearson)

         T7       T8
T7  1.00000  0.96317
T8  0.96317  1.00000


In [10]:
srd_df = new_df.drop(["P3", "P4", "P10", "T8"], axis=1)
srd_df["SRD"] = 3.6 * srd_df["U7"] / srd_df["F10"]
srd_df = srd_df.drop(["U7", "F10"], axis=1)

if not os.path.exists("../data/processed/SRD_Lysekil.pkl"):
    srd_df.to_pickle("../data/processed/SRD_Lysekil.pkl")

pearson = srd_df.iloc[:, 1:].corr(method="pearson")
visualize.make_heatmap(pearson)